In [25]:
from pathlib import Path
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

## Loading training images

In [26]:
# Path to folders with training data
lace_path = Path("style_image_clean") / "lace"
graphic_path = Path("style_image_clean") / "graphic"
floral_path = Path("style_image_clean") / "floral"
stripes_path = Path("style_image_clean") / "stripes"
pattern_path = Path("style_image_clean") / "pattern"
pocket_path = Path("style_image_clean") / "pocket"
print_path = Path("style_image_clean") / "print"
v_neckline_path = Path("style_image_clean") / "v-neckline"

images = []
labels = []

In [27]:
# Load all the lace images
for img in lace_path.glob("*.jpg"):
    img = image.load_img(img)
    image_array = image.img_to_array(img)
    images.append(image_array)

    # For each 'lace' image, label as 0
    labels.append(0)

In [28]:
# Load all the graphic images
for img in graphic_path.glob("*.jpg"):
    img = image.load_img(img)
    image_array = image.img_to_array(img)
    images.append(image_array)

    # For each 'graphic' image, label as 1
    labels.append(1)

In [29]:
# Load all the floral images
for img in floral_path.glob("*.jpg"):
    img = image.load_img(img)
    image_array = image.img_to_array(img)
    images.append(image_array)

    # For each 'floral' image, label as 2
    labels.append(2)

In [30]:
# Load all the stripes images
for img in stripes_path.glob("*.jpg"):
    img = image.load_img(img)
    image_array = image.img_to_array(img)
    images.append(image_array)

    # For each 'stripes' image, label as 3
    labels.append(3)

In [31]:
# Load all the pattern images
for img in pattern_path.glob("*.jpg"):
    img = image.load_img(img)
    image_array = image.img_to_array(img)
    images.append(image_array)

    # For each 'pattern' image, label as 4
    labels.append(4)

In [32]:
# Load all the pocket images
for img in pocket_path.glob("*.jpg"):
    img = image.load_img(img)
    image_array = image.img_to_array(img)
    images.append(image_array)

    # For each 'pocket' image, label as 5
    labels.append(5)

In [33]:
# Load all the print images
for img in print_path.glob("*.jpg"):
    img = image.load_img(img)
    image_array = image.img_to_array(img)
    images.append(image_array)

    # For each 'print' image, label as 6
    labels.append(6)

In [34]:
# Load all the v-neckline images
for img in v_neckline_path.glob("*.jpg"):
    img = image.load_img(img)
    image_array = image.img_to_array(img)
    images.append(image_array)

    # For each 'print' image, label as 7
    labels.append(7)

In [36]:
from sklearn.utils import shuffle
images, labels = shuffle(images, labels)

In [55]:
X = np.array(images)
y = np.array(labels)

from tensorflow.keras.utils import to_categorical
y_data=to_categorical(y,num_classes=8)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_data, test_size=0.33, random_state=42)

In [85]:
X_train.shape

(3654, 256, 256, 3)

In [91]:
y

array([1, 7, 6, ..., 6, 3, 3])

In [57]:
X_train = vgg16.preprocess_input(X_train)
X_test = vgg16.preprocess_input(X_test)

In [59]:
# Load a pre-trained neural network to use as a feature extractor
# set include_top = False to remove the top layer to be replaced by our own classifier
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [60]:
base_model.summary()
#no flatten / ANN layers, coz include_top=False

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [61]:
# Define the layers in the new classification prediction 
from tensorflow.keras.models import Model
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)        
x = Dense(128, activation='relu')(x)        
x = Dense(64, activation='relu')(x)        
x = Dropout(0.2)(x)
predictions = Dense(8, activation='softmax')(x)  

# Define trainable model which links input from the  base model to the new classification prediction layers
model = Model(inputs=base_model.input, outputs=predictions)

print (model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [62]:
#   Freeze all layers in the vgg16 base model 

for layer in base_model.layers:
    print(layer)
    layer.trainable = False
#   Define model compile for basic Transfer Learning
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Performing Transfer Learning


In [64]:
# Train the model
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=8,
    validation_data=(X_test,y_test)
)

Epoch 1/5
457/457 [==============================] - 972s 2s/step - loss: 1.4654 - accuracy: 0.4661 - val_loss: 1.6096 - val_accuracy: 0.4153
Epoch 2/5
457/457 [==============================] - 967s 2s/step - loss: 1.3955 - accuracy: 0.4893 - val_loss: 1.5824 - val_accuracy: 0.4936
Epoch 3/5
457/457 [==============================] - 957s 2s/step - loss: 1.2991 - accuracy: 0.5252 - val_loss: 1.6174 - val_accuracy: 0.4853
Epoch 4/5
457/457 [==============================] - 963s 2s/step - loss: 1.2401 - accuracy: 0.5435 - val_loss: 1.5343 - val_accuracy: 0.5192
Epoch 5/5
457/457 [==============================] - 971s 2s/step - loss: 1.1678 - accuracy: 0.5761 - val_loss: 1.5227 - val_accuracy: 0.4720


## Tuning -  Using ImageDataGenerator

In [82]:
gen = ImageDataGenerator(rotation_range=20, zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,horizontal_flip=True)

H = model.fit_generator(gen.flow(X_train, y_train, batch_size=8),
                        validation_data=(X_test,y_test), 
                        steps_per_epoch=len(X_train) // 8, 
                        epochs=30,
                        callbacks=[early_stop])

/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
456/456 [==============================] - 927s 2s/step - loss: 1.7840 - accuracy: 0.3683 - val_loss: 1.4568 - val_accuracy: 0.5053
Epoch 2/30
456/456 [==============================] - 909s 2s/step - loss: 1.6102 - accuracy: 0.4221 - val_loss: 1.4563 - val_accuracy: 0.5158
Epoch 3/30
456/456 [==============================] - 917s 2s/step - loss: 1.5567 - accuracy: 0.4493 - val_loss: 1.3871 - val_accuracy: 0.5292
Epoch 4/30
456/456 [==============================] - 963s 2s/step - loss: 1.5421 - accuracy: 0.4509 - val_loss: 1.4190 - val_accuracy: 0.5103
Epoch 5/30
456/456 [==============================] - 1030s 2s/step - loss: 1.4955 - accuracy: 0.4649 - val_loss: 1.3778 - val_accuracy: 0.5230
Epoch 6/30
456/456 [==============================] - 893s 2s/step - loss: 1.4890 - accuracy: 0.4841 - val_loss: 1.4343 - val_accuracy: 0.5336
Epoch 7/30
456/456 [==============================] - 892s 2s/step - loss: 1.4644 - accuracy: 0.4660 - val_loss: 1.4939 - val_accuracy: 0.524

In [96]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 

Saved model to disk
